# Read architecture from RSML file and run a simulation
This is the notebook version of the example in read the doc. 
The notebook illustrate how to compute hydraulic on a root system architecture.

Here, we load arabidopsis RSA and compute hydraulic fluxes ar any point of the architecture.
Radial and axial conductances are defined based on measurements (anatomy for axial conductance and pressure-probe for radial conductance).

The method is describe in https://hydroroot.rtfd.io

The paper is here : https://doi.org/10.1093/plphys/kiac281

Point to the source files if the notebook is run locally, from a git repository clone for example, without hydroroot installation, but only the dependencies installed.

In [1]:
import rsml
from hydroroot import radius
from hydroroot.main import hydroroot_flow
from oawidgets.plantgl import PlantGL
from oawidgets.mtg import plot as plot_mtg
from hydroroot.display import mtg_scene
from hydroroot.hydro_io import import_rsml_to_discrete_mtg, export_mtg_to_rsml
from pathlib import Path
from pprint import pprint
%run read.py

In [2]:
data = Path('data')/'UC1_HIRROS_arabidopsis'
rsa_files = list(data.glob('*.rsml'))
pprint(rsa_files)

[PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN016.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN017.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN021.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN026.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN007.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN028.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN033.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN005.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN013.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN015.rsml')]


Read the RSML file and convert it into a *continuous* MTG. This is a MTG where each root (primary and lateral) is represented by one vertex. The geometry of each root is then stored in g_c.property('geometry').

In [3]:
g = rsml.rsml2mtg(rsa_files[0])

To be used in HydroRoot the MTG has to be converted to a *discrete* form of MTG, i.e. each vertex represent a representative elementary volume of a given length for example $10^{-4}$ m. In HydroRoot the lengths are in meter, therefore we must retrieve the resolution and the unit of the RSML file. 

In [4]:
resolution = g.graph_properties()['metadata']['resolution'] # pixel to unit
unit = g.graph_properties()['metadata']['unit']
print(unit)

pixel(um)


In [5]:
resolution = resolution * 1e-6 # pixel to unit to m

In [6]:
# Split the forest into 5 independent graphs
g1, g2, g3, g4, g5 = gs =  split(g)

Build the discrete MTG

In [7]:
g = import_rsml_to_discrete_mtg(g1, segment_length = 1.0e-4, resolution = resolution)

In [8]:
plot_mtg(g)

mtg.html


Calculate some properties needed to simulate a sap flux from the root under overpressure. 

Set the radius with values inspired from Boursiac et al. 2022 https://doi.org/10.1093/plphys/kiac281

In [9]:
g = radius.ordered_radius(g, 7.0e-5, 0.7) # root radii
g = radius.compute_relative_position(g) # Compute the position of each segment relative to the axis bearing it

Some conductance data versus distance to tip, values inspired from Boursiac et al. 2022 https://doi.org/10.1093/plphys/kiac281

In [10]:
k_radial_data=([0, 0.2],[330.0,330.0]) #unit: microL/(s.MPa.m**2)
K_axial_data=([0, 1.5e-2, 4.0e-2, 6.0e-2, 8.0e-2, 10.0e-2, 12.0e-2, 15.0e-2, 20.0e-2]
              ,[ 3.0E-05, 3.0E-04, 3.9E-04, 3.5E-04, 4.0E-04, 8.0E-04, 9.0E-04, 8.4E-04, 4.4E-04]) #microL.m/(s.Mpa)

Flux and equivalent conductance calculation, for a root in an external hydroponic medium at 0.4 MPa, its base at 0.1 MPa, and with the conductances set above.

In [11]:
g, keq, jv = hydroroot_flow(g, psi_e = 0.4, psi_base = 0.1, axial_conductivity_data = K_axial_data, radial_conductivity_data = k_radial_data)

In [12]:
print('equivalent root conductance (microL/s/MPa): ',keq, 'sap flux (microL/s): ', jv)

equivalent root conductance (microL/s/MPa):  0.001892374763559223 sap flux (microL/s):  0.000567712429067767


Display the local water uptake heatmap in 3D

In [20]:
s = mtg_scene(g, prop_cmap = 'psi_out') # create a scene from the mtg with the property j is the radial flux in ul/s
PlantGL(s) # display the root into the plantgl oawidget

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

Export the MTG to RSML

At this stage (2022-08-22) only the root length and the branching position are used to simulate architecture in hydroponic solution. The exact position in 3D is not stored in the discrete MTG form and so not exported to RMSL.

In [13]:
#export_mtg_to_rsml(g, "test.rsml", segment_length = 1.0e-4)

TODO: time-sequence


The resolution of the RSML data is 1.0e-4 and the unit is meter.

In [18]:
g.property_names()

['edge_type',
 'label',
 'base_length',
 'length',
 'order',
 'position',
 'radius',
 'relative_position',
 'K_exp',
 'K',
 'k0',
 'k',
 'Keq',
 'psi_in',
 'psi_out',
 'j',
 'J_out',
 'color',
 'keq',
 'J']

In [19]:
g[3]

{'edge_type': '<',
 'label': 'S',
 'length': 0.0001,
 'order': 0,
 'position': 0.005200000000000001,
 'radius': 9.814814814814793e-05,
 'relative_position': 0.9629629629629629,
 'K_exp': 0.0001236,
 'K': 1.236,
 'k0': 330.0,
 'k': 1.4514158059584844e-05,
 'Keq': 0.0018689907351778657,
 'psi_in': 0.10134785322992496,
 'psi_out': 0.10089556891718675,
 'j': 4.3346844630552006e-06,
 'J_out': 0.0005590234105444243,
 'color': [141, 0, 0],
 'vid': 3,
 'index': 3,
 'complex': 0,
 'parent': 2,
 'scale': 1}